In [522]:
%run Importaciones.ipynb
%run Variables_Clases.ipynb

#

In [523]:
experimento_1_nor = pd.read_csv('../datos/experimento_1_nor.csv', sep=',')
experimento_2_nor = pd.read_csv('../datos/experimento_2_nor.csv', sep=',')
experimento_3_nor = pd.read_csv('../datos/experimento_3_nor.csv', sep=',')
experimento_4_nor = pd.read_csv('../datos/experimento_4_nor.csv', sep=',')

experimentos_nor = [experimento_1_nor, experimento_2_nor, experimento_3_nor,experimento_4_nor]



In [524]:
experimento_1_nor.head()

,Timestamp,Attention,Meditation,Delta,Theta,LowAlpha,HighAlpha,LowBeta,HighBeta,LowGamma,HighGamma,Key
0,54127,0.945946,0.267606,0.021185,0.079462,0.061727,0.076266,0.100597,0.548819,0.751252,0.856338,4
1,55056,0.864865,0.436620,0.003739,0.027162,0.164694,0.255753,0.034888,0.340997,0.461066,0.450235,0
2,57043,0.635135,0.816901,0.049114,0.069893,0.113457,0.095536,0.189586,0.584612,1.000000,1.000000,0
3,58203,0.310811,0.774648,0.018012,0.420046,0.210478,0.178815,0.294905,0.059359,0.270947,0.219875,0
4,59952,0.459459,0.478873,0.001672,0.006495,0.009676,0.000955,0.063348,0.080122,0.261384,0.026917,0


# holdout

Conjunto de entrenamiento: 81% del total
Conjunto de validación: 9% del total
Conjunto de prueba: 10% del total

In [549]:
tests_holdout= []
matriz_confusion =0


def evaluar_holdout(dataframe):
    
    y = dataframe['Key'].values
    X = dataframe.drop(columns=['Key']).values
    
    # Se dividen los datos. X_train_val sera el 90% de los datos y X_test el 10%
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=SEED, stratify=y, train_size=TRAIN)

    # Se dividen los datos. X_train sera el 90% de los datos de X_train_val y X_val el restante 10%
    X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=SEED, stratify=y, train_size=TRAIN)
    
    tests_holdout.append(X_test)
    tests_holdout.append(y_test)
    
    # Entreno el modelo
    model_KNN.fit(X_train, y_train)
    model_TREE.fit(X_train, y_train)
    model_RANDOM.fit(X_train, y_train)
    
    # Evaluo el modelo con X_val
    y_predict_knn = model_KNN.predict(X_val)
    y_predict_tree = model_TREE.predict(X_val)
    y_predict_random = model_RANDOM.predict(X_val)
    
    # Evaluo el rendimiento que ha tenido Validation 
    knn_ac = accuracy_score(y_val, y_predict_knn)
    tree_ac = accuracy_score(y_val, y_predict_tree)
    random_ac = accuracy_score(y_val, y_predict_random)
    
    matriz_confusion = confusion_matrix(y_val, y_predict_knn)
    
    
    return ((knn_ac), (tree_ac), (random_ac))
    
    

In [ ]:
class_names1 = exp['Key'].unique()

conf_mat_exp1 = pd.DataFrame(modelos_all[0], index=class_names1, columns=class_names)

In [526]:
modelos_all= []

for exp in experimentos_nor:
    
    modelos_all.append(evaluar_holdout(exp))

datos_holdout = pd.DataFrame(modelos_all, columns=['Knn_holdout', 'Tree_holdout', 'Random_holdout'])
#creo los valores nuevo para los indices de las filas 
nuevos_indices = {0: 'Exp_1', 1: 'Exp_2', 2: 'Exp_3', 3: 'Exp_4'}
# combio los valores anteriores en el dataframene con rename
datos_holdout = datos_holdout.rename(index=nuevos_indices)

In [527]:
len(tests_holdout)

8

In [528]:
display(datos_holdout)

,Knn_holdout,Tree_holdout,Random_holdout
Exp_1,0.923077,0.769231,0.769231
Exp_2,0.937500,0.812500,0.437500
Exp_3,0.800000,0.700000,0.450000
Exp_4,0.800000,0.750000,0.450000


 # k-fold cross-validation

In [529]:
#ACC_SCORER

def evaluar_cross_val(dataframe):
    
    knn_means = []
    tree_means = []
    random_means =[]
    
    y = dataframe['Key'].values
    X = dataframe.drop(columns=['Key']).values
     
    # Calculo las  metricas       
    knn = cross_val_score(model_KNN, X, y, cv=CV, scoring=ACC_SCORER)      
    tree = cross_val_score(model_TREE, X, y, cv=CV, scoring=ACC_SCORER)  
    random = cross_val_score(model_RANDOM, X, y, cv=CV, scoring=ACC_SCORER)

    # Calculo la media de las metricas anteriores
    knn_mean = knn.mean()
    tree_mean = tree.mean()
    random_mean = random.mean()

    return ((knn_mean), (tree_mean), (random_mean))
  


In [530]:
modelos_all= []

for exp in experimentos_nor:
    
    modelos_all.append(evaluar_cross_val(exp))

datos_cross_val = pd.DataFrame(modelos_all, columns=['Knn_cross_val', 'Tree_cross_val', 'Random_cross_val'])
#creo los valores nuevo para los indices de las filas 
nuevos_indices = {0: 'Exp_1', 1: 'Exp_2', 2: 'Exp_3', 3: 'Exp_4'}
# combio los valores anteriores en el dataframene con rename
datos_cross_val = datos_cross_val.rename(index=nuevos_indices)

In [531]:
display(datos_cross_val)

,Knn_cross_val,Tree_cross_val,Random_cross_val
Exp_1,0.921429,0.869048,0.530952
Exp_2,0.838393,0.846429,0.513393
Exp_3,0.850000,0.749444,0.426111
Exp_4,0.773333,0.700556,0.301667


# leave-one-out

In [532]:
#ACC_SCORER

def evaluar_loo(dataframe):
    
    knn_means = []
    tree_means = []
    random_means =[]
    
    y = dataframe['Key'].values
    X = dataframe.drop(columns=['Key']).values
     
    # Calculo las  metricas       
    knn = cross_val_score(model_KNN, X, y, cv=LOO, scoring=ACC_SCORER)      
    tree = cross_val_score(model_TREE, X, y, cv=LOO, scoring=ACC_SCORER)  
    random = cross_val_score(model_RANDOM, X, y, cv=LOO, scoring=ACC_SCORER)

    # Calculo la media de las metricas anteriores
    knn_mean = knn.mean()
    tree_mean = tree.mean()
    random_mean = random.mean()

    return ((knn_mean), (tree_mean), (random_mean))

In [533]:
modelos_all= []

for exp in experimentos_nor:
    
    modelos_all.append(evaluar_loo(exp))

datos_loo = pd.DataFrame(modelos_all, columns=['Knn_loo', 'Tree_loo', 'Random_loo'])
#creo los valores nuevo para los indices de las filas 
nuevos_indices = {0: 'Exp_1', 1: 'Exp_2', 2: 'Exp_3', 3: 'Exp_4'}
# combio los valores anteriores en el dataframene con rename
datos_loo = datos_loo.rename(index=nuevos_indices)

In [534]:
display(datos_loo)

,Knn_loo,Tree_loo,Random_loo
Exp_1,0.930769,0.853846,0.500000
Exp_2,0.929936,0.917197,0.503185
Exp_3,0.927461,0.725389,0.523316
Exp_4,0.873737,0.792929,0.444444


In [535]:


df_concatenado = pd.concat([datos_holdout, datos_cross_val, datos_loo], axis=1)


display(df_concatenado)


,Knn_holdout,Tree_holdout,Random_holdout,Knn_cross_val,Tree_cross_val,Random_cross_val,Knn_loo,Tree_loo,Random_loo
Exp_1,0.923077,0.769231,0.769231,0.921429,0.869048,0.530952,0.930769,0.853846,0.500000
Exp_2,0.937500,0.812500,0.437500,0.838393,0.846429,0.513393,0.929936,0.917197,0.503185
Exp_3,0.800000,0.700000,0.450000,0.850000,0.749444,0.426111,0.927461,0.725389,0.523316
Exp_4,0.800000,0.750000,0.450000,0.773333,0.700556,0.301667,0.873737,0.792929,0.444444


In [536]:

#primer mayor valor en verde
#segundo mayor valor en amarillo 


def colorear_mayores(fila):
    mayores_indices = fila.nlargest(2).index  # Obtener los índices de los tres valores más grandes en la fila
    
    primer_mayor = mayores_indices[0]
    segundo_mayor = mayores_indices[1]
    
    estilos = []
    for columna in fila.index:

        if columna == primer_mayor:
            estilo = 'background-color: green'
        elif columna == segundo_mayor:
            estilo = 'background-color: yellow'
        else:
            estilo = ''
        estilos.append(estilo)
    
    return estilos

comparativa_ml = df_concatenado.style.apply(colorear_mayores, axis=1)

display(comparativa_ml)



,Knn_holdout,Tree_holdout,Random_holdout,Knn_cross_val,Tree_cross_val,Random_cross_val,Knn_loo,Tree_loo,Random_loo
Exp_1,0.923077,0.769231,0.769231,0.921429,0.869048,0.530952,0.930769,0.853846,0.500000
Exp_2,0.937500,0.812500,0.437500,0.838393,0.846429,0.513393,0.929936,0.917197,0.503185
Exp_3,0.800000,0.700000,0.450000,0.850000,0.749444,0.426111,0.927461,0.725389,0.523316
Exp_4,0.800000,0.750000,0.450000,0.773333,0.700556,0.301667,0.873737,0.792929,0.444444


# cross validation

In [537]:
def evaluar_cross_validation(dataframe):
    
    aciertos = []
    y = dataframe['Key'].values
    X = dataframe.drop(columns=['Key']).values
    
    
    X1, X2, y1, y2 = train_test_split(X, y, random_state=SEED, train_size=0.5)

    
    y2_model = model_KNN.fit(X1, y1).predict(X2)
    y1_model = model_KNN.fit(X2, y2).predict(X1)
    aciertos.append(accuracy_score(y1, y1_model))
    aciertos.append(accuracy_score(y2, y2_model))
    media = np.mean(aciertos)
    aciertos.append(media)
 
    return aciertos

In [538]:
modelos_all= []

for exp in experimentos_nor:
    
    modelos_all.append(evaluar_cross_validation(exp))   

datos_cross_validation = pd.DataFrame(modelos_all, columns=['Knn_cross_validation_A','Knn_cross_validation_B','Knn_cross_validation_Media'])
#creo los valores nuevo para los indices de las filas 
nuevos_indices = {0: 'Exp_1', 1: 'Exp_2', 2: 'Exp_3', 3: 'Exp_4'}
# combio los valores anteriores en el dataframene con rename
datos_cross_validation = datos_cross_validation.rename(index=nuevos_indices)


In [539]:
display(datos_cross_validation)

,Knn_cross_validation_A,Knn_cross_validation_B,Knn_cross_validation_Media
Exp_1,0.830769,0.861538,0.846154
Exp_2,0.769231,0.822785,0.796008
Exp_3,0.843750,0.793814,0.818782
Exp_4,0.666667,0.737374,0.702020


# cross_val_predict

In [540]:
def evaluar_cross_val_predict(dataframe):
    
    y = dataframe['Key'].values
    X = dataframe.drop(columns=['Key']).values
        
    predicciones = cross_val_predict(model_KNN, X, y, cv=CV)
    matriz_confusion = confusion_matrix(y, predicciones)
    #reporte = classification_report(y, predicciones)
    #precision, recall, f1_score, _ = reporte.split('\n')[-2].split()[1:]#
    
    return ((matriz_confusion))

In [541]:
modelos_all= []
class_names = []

for exp in experimentos_nor:
    
    modelos_all.append(evaluar_cross_val_predict(exp))   
    class_names = exp['Key'].unique()

conf_mat_exp1 = pd.DataFrame(modelos_all[0], index=class_names, columns=class_names)
conf_mat_exp2 = pd.DataFrame(modelos_all[1], index=class_names, columns=class_names)
conf_mat_exp3 = pd.DataFrame(modelos_all[2], index=class_names, columns=class_names)
conf_mat_exp4 = pd.DataFrame(modelos_all[3], index=class_names, columns=class_names)


In [542]:
display (conf_mat_exp1)
display (conf_mat_exp2)
display (conf_mat_exp3)
display (conf_mat_exp4)

,4,0,1,3,2
4,30,1,3,2,3
0,0,22,0,0,0
1,0,0,23,0,0
3,0,0,0,22,0
2,1,0,0,0,23


,4,0,1,3,2
4,21,3,6,3,7
0,2,27,0,0,0
1,2,0,26,0,0
3,1,0,0,29,0
2,1,0,0,0,29


,4,0,1,3,2
4,34,9,4,2,4
0,2,43,0,0,0
1,1,0,30,0,0
3,3,0,0,28,0
2,3,1,0,0,29


,4,0,1,3,2
4,27,8,7,4,8
0,4,40,0,0,0
1,3,0,29,0,0
3,6,0,0,22,0
2,4,1,0,0,35


# Regresion Linear   N/A

In [543]:

def evaluar_linear_regresion(dataframe):
    
    y = dataframe['Key'].values
    X = dataframe.drop(columns=['Key']).values
        
    model_LINEAR.fit(X, y)

    evaluacion = model_LINEAR.score(X, y)
    
    return ((evaluacion))


In [544]:
modelos_all= []

for exp in experimentos_nor:
    
    modelos_all.append(evaluar_linear_regresion(exp))   

datos_linear_regresion = pd.DataFrame(modelos_all, columns=['Linear_regresion'])
#creo los valores nuevo para los indices de las filas 
nuevos_indices = {0: 'Exp_1', 1: 'Exp_2', 2: 'Exp_3', 3: 'Exp_4'}
# combio los valores anteriores en el dataframene con rename
datos_linear_regresion = datos_linear_regresion.rename(index=nuevos_indices)

In [545]:
display(datos_linear_regresion)

,Linear_regresion
Exp_1,0.315697
Exp_2,0.169613
Exp_3,0.258738
Exp_4,0.076861


In [ ]:


# probar regresion logistica

